In [1]:
import dask.array as da
import numpy as np

x = np.arange(64).reshape((8, 8))
d = da.from_array(x, chunks=(4, 4))
d.compute()

array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29, 30, 31],
       [32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47],
       [48, 49, 50, 51, 52, 53, 54, 55],
       [56, 57, 58, 59, 60, 61, 62, 63]])

In [2]:
g = da.overlap.overlap(d, depth={0: 2, 1: 1}, boundary={0: 1, 1: 0})
# g = da.overlap.overlap(d, depth={1: 1}, boundary={0: 1, 1: 0})
g.compute()

array([[ 0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  0],
       [ 0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  0],
       [ 0,  0,  1,  2,  3,  4,  3,  4,  5,  6,  7,  0],
       [ 0,  8,  9, 10, 11, 12, 11, 12, 13, 14, 15,  0],
       [ 0, 16, 17, 18, 19, 20, 19, 20, 21, 22, 23,  0],
       [ 0, 24, 25, 26, 27, 28, 27, 28, 29, 30, 31,  0],
       [ 0, 32, 33, 34, 35, 36, 35, 36, 37, 38, 39,  0],
       [ 0, 40, 41, 42, 43, 44, 43, 44, 45, 46, 47,  0],
       [ 0, 16, 17, 18, 19, 20, 19, 20, 21, 22, 23,  0],
       [ 0, 24, 25, 26, 27, 28, 27, 28, 29, 30, 31,  0],
       [ 0, 32, 33, 34, 35, 36, 35, 36, 37, 38, 39,  0],
       [ 0, 40, 41, 42, 43, 44, 43, 44, 45, 46, 47,  0],
       [ 0, 48, 49, 50, 51, 52, 51, 52, 53, 54, 55,  0],
       [ 0, 56, 57, 58, 59, 60, 59, 60, 61, 62, 63,  0],
       [ 0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  0],
       [ 0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  0]])

In [3]:
print(f'original chunks: {d.chunks}')
print(f'new chunks: {g.chunks}')

original chunks: ((4, 4), (4, 4))
new chunks: ((8, 8), (6, 6))


In [4]:
def myfunc(block, block_id=None):
    print(block_id)
    print(block)
    return np.zeros([1]*len(block_id)) # return should match g.shape

In [5]:
g.map_blocks(myfunc, dtype='uint8', chunks=(1, 1)).compute()

(0, 0)
[[ 0  1  1  1  1  1]
 [ 0  1  1  1  1  1]
 [ 0  0  1  2  3  4]
 [ 0  8  9 10 11 12]
 [ 0 16 17 18 19 20]
 [ 0 24 25 26 27 28]
 [ 0 32 33 34 35 36]
 [ 0 40 41 42 43 44]]
(1, 0)
[[ 0 16 17 18 19 20]
 [ 0 24 25 26 27 28]
 [ 0 32 33 34 35 36]
 [ 0 40 41 42 43 44]
 [ 0 48 49 50 51 52]
 [ 0 56 57 58 59 60]
 [ 0  1  1  1  1  1]
 [ 0  1  1  1  1  1]]
(1, 1)
[[19 20 21 22 23  0]
 [27 28 29 30 31  0]
 [35 36 37 38 39  0]
 [43 44 45 46 47  0]
 [51 52 53 54 55  0]
 [59 60 61 62 63  0]
 [ 1  1  1  1  1  0]
 [ 1  1  1  1  1  0]]
(0, 1)
[[ 1  1  1  1  1  0]
 [ 1  1  1  1  1  0]
 [ 3  4  5  6  7  0]
 [11 12 13 14 15  0]
 [19 20 21 22 23  0]
 [27 28 29 30 31  0]
 [35 36 37 38 39  0]
 [43 44 45 46 47  0]]


array([[0., 0.],
       [0., 0.]])

In [6]:
result = da.overlap.trim_internal(g, {0: 2, 1: 1})

In [7]:
result.compute()

array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29, 30, 31],
       [32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47],
       [48, 49, 50, 51, 52, 53, 54, 55],
       [56, 57, 58, 59, 60, 61, 62, 63]])